In [1]:
import cv2
import numpy as np
import tensorflow
import keras
from keras import layers as L
import ssl
import os

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def build_model(input_shape, num_classes=3):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Backbone """
    backbone = keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        alpha=1.0
    )
    backbone.trainable = True
    # backbone.summary()

    """ Detection Head """
    x = backbone.output
    x = L.Conv2D(256, kernel_size=1, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.5)(x)
    bbox = L.Dense(4, activation="sigmoid", name="bbox")(x)
    label = L.Dense(num_classes, activation="softmax", name="label")(x)

    """ Model """
    model = keras.models.Model(inputs=[inputs], outputs=[bbox, label])
    return model

In [4]:
def load_model():
    model_path = "./model.weights.h5"
    
    model = build_model((218, 384, 3))
    model.load_weights(model_path)

    return model

In [1]:
def show_video(model):
    vid = cv2.VideoCapture(0)
    
    while True:
        _, frame = vid.read()
        grey = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        _, binary_result = cv2.threshold(grey, 127, 255, cv2.THRESH_BINARY) 
        cv2.imwrite("thing/img.jpg", cv2.resize(binary_result, (384, 218)))
        img = cv2.imread("thing/img.jpg")
        os.remove("thing/img.jpg")

        x = cv2.resize(img, (384, 218))
        x = (x - 127.5) / 127.5
        x = np.expand_dims(x, axis=0)

        pred_bbox, label = model.predict(x, verbose=0)
        
        label_found = ""
        if label[0][0] >= label[0][1] and label[0][0] >= label[0][2]:
            label_found = "H"
        elif label[0][1] >= label[0][0] and label[0][1] >= label[0][2]:
            label_found = "S"
        elif label[0][2] >= label[0][0] and label[0][2] >= label[0][1]:
            label_found = "U"

        max_confidence = max(label[0][0], label[0][1], label[0][2])
        if not max_confidence >= .85:
            label_found = "None"

        
        print(pred_bbox, label, label_found, max_confidence)

        cv2.imshow("Normal", cv2.resize(frame, (384, 218)))
        cv2.imshow("Letter Mask", cv2.resize(binary_result, (384, 218)))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    vid.release()
    cv2.destroyAllWindows()

In [6]:
def main():
    model = load_model()
    show_video(model)

In [7]:
main()

/var/folders/4w/7mmrnj315pzgcqvpx7n96sf40000gn/T/ipykernel_64269/2947174705.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = keras.applications.MobileNetV2(


[[0.60566574 0.5106019  0.6816279  0.66886276]] [[0.25144404 0.50306416 0.24549177]] None 0.50306416
[[0.5877096 0.491243  0.6782181 0.6703168]] [[0.24984728 0.5594078  0.19074498]] None 0.5594078
[[0.5614193  0.5560425  0.6503364  0.72973394]] [[0.37545842 0.4433375  0.18120407]] None 0.4433375
[[0.54615796 0.57344407 0.64669937 0.7390575 ]] [[0.33982703 0.4354715  0.22470139]] None 0.4354715
[[0.5786611 0.5632307 0.6602045 0.7355585]] [[0.36010322 0.47025353 0.16964324]] None 0.47025353
[[0.6377348  0.55482394 0.72365826 0.73531437]] [[0.3280476  0.48740405 0.18454833]] None 0.48740405
[[0.618783  0.5650889 0.7058462 0.7548183]] [[0.17420144 0.6951617  0.13063684]] None 0.6951617
[[0.6324054 0.592638  0.7263408 0.7894874]] [[0.19854322 0.5544876  0.24696925]] None 0.5544876
[[0.68512493 0.5408714  0.770074   0.7516866 ]] [[0.20472923 0.34441942 0.45085132]] None 0.45085132
[[0.5994975  0.5551162  0.69500506 0.7475062 ]] [[0.18268923 0.40152937 0.41578138]] None 0.41578138
[[0.6313574

KeyboardInterrupt: 